# This is from scratch version with standard reward 

In [ ]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gymnasium as gym
import os

# Create the HalfCheetah environment
env = gym.make("HalfCheetah-v5", render_mode=None)


# Get state and action dimensions from the environment
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

# Set up device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the Critic network (QNet)
class QNet(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=64):
        super().__init__()
        # Input dimension: state_dim + action_dim
        self.hidden = nn.Linear(state_dim + action_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, 1)

    def forward(self, s, a):
        x = torch.cat((s, a), dim=-1)
        x = self.hidden(x)
        x = F.relu(x)
        return self.output(x)

# Define the Actor network (PolicyNet)
class PolicyNet(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=64):
        super().__init__()
        # Input dimension: state_dim; output dimension: action_dim
        self.hidden = nn.Linear(state_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, action_dim)

    def forward(self, s):
        x = self.hidden(s)
        x = F.relu(x)
        x = self.output(x)
        return torch.tanh(x)

# Initialize networks and target networks.
q_origin_model = QNet(state_dim, action_dim).to(device)
q_target_model = QNet(state_dim, action_dim).to(device)
_ = q_target_model.requires_grad_(False)

mu_origin_model = PolicyNet(state_dim, action_dim).to(device)
mu_target_model = PolicyNet(state_dim, action_dim).to(device)
_ = mu_target_model.requires_grad_(False)

# Hyperparameters and optimizers
gamma = 0.99
opt_q = torch.optim.AdamW(q_origin_model.parameters(), lr=0.0005)
opt_mu = torch.optim.AdamW(mu_origin_model.parameters(), lr=0.0005)

def optimize(states, actions, rewards, next_states, dones):
    # Convert lists to tensors
    states = torch.tensor(np.stack(states), dtype=torch.float).to(device)

    actions = torch.tensor(actions, dtype=torch.float).to(device)
    rewards = torch.tensor(rewards, dtype=torch.float).unsqueeze(dim=1).to(device)
    next_states = torch.tensor(next_states, dtype=torch.float).to(device)
    dones = torch.tensor(dones, dtype=torch.float).unsqueeze(dim=1).to(device)

    # Compute target for critic: Q_target = r + gamma * Q_target(s', mu_target(s'))
    with torch.no_grad():
        next_actions = mu_target_model(next_states)
        target = rewards + (1 - dones) * gamma * q_target_model(next_states, next_actions)

    # Critic loss
    q_value = q_origin_model(states, actions)
    q_loss = F.mse_loss(q_value, target.detach())
    opt_q.zero_grad()
    q_loss.backward()
    opt_q.step()

    # Actor loss: maximize Q(s, mu(s)) -> minimizing -Q(s, mu(s))
    mu_value = mu_origin_model(states)
    q_value_for_mu = q_origin_model(states, mu_value)
    mu_loss = -q_value_for_mu.mean()
    opt_mu.zero_grad()
    mu_loss.backward()
    opt_mu.step()

    # Ensure gradients remain enabled for subsequent iterations.
    for p in q_origin_model.parameters():
        p.requires_grad = True

    return q_loss.item(), mu_loss.item()

tau = 0.002
def update_target():
    # Soft update target networks.
    for var, var_target in zip(q_origin_model.parameters(), q_target_model.parameters()):
        var_target.data = tau * var.data + (1.0 - tau) * var_target.data
    for var, var_target in zip(mu_origin_model.parameters(), mu_target_model.parameters()):
        var_target.data = tau * var.data + (1.0 - tau) * var_target.data

# Replay Buffer to store experiences
class ReplayBuffer:
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size
        self.buffer = []

    def add(self, item):
        if len(self.buffer) == self.buffer_size:
            self.buffer.pop(0)
        self.buffer.append(item)

    def sample(self, batch_size):
        items = random.sample(self.buffer, batch_size)
        states   = [i[0] for i in items]
        actions  = [i[1] for i in items]
        rewards  = [i[2] for i in items]
        n_states = [i[3] for i in items]
        dones    = [i[4] for i in items]
        return states, actions, rewards, n_states, dones

    def length(self):
        return len(self.buffer)

buffer = ReplayBuffer(buffer_size=20000)

# Ornstein-Uhlenbeck noise for exploration.
class OrnsteinUhlenbeckActionNoise:
    def __init__(self, mu, sigma, theta=0.15, dt=1e-2, x0=None):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
            self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        self.x_prev = x
        return x

    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

ou_action_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(action_dim), sigma=np.ones(action_dim) * 0.05)

def pick_sample(s):
    with torch.no_grad():
        s_np = np.array(s)
        s_batch = np.expand_dims(s_np, axis=0)
        s_batch = torch.tensor(s_batch, dtype=torch.float).to(device)
        action_det = mu_origin_model(s_batch).squeeze(0)
        noise = ou_action_noise()
        action = action_det.cpu().numpy() + noise
        action = np.clip(action, -1.0, 1.0)
        return action

# Training parameters
batch_size = 250
num_episodes = 2000

# Prepare logging file
log_file_path = "training_log.txt"
log_file = open(log_file_path, "w")
log_file.write("episode,reward,avg_q_loss,avg_mu_loss\n")

if __name__ == '__main__':
    reward_records = []
    for ep in range(num_episodes):
        s, _ = env.reset()
        done = False
        cum_reward = 0
        # For accumulating losses over the episode.
        episode_q_loss = 0
        episode_mu_loss = 0
        training_steps = 0

        while not done:
            a = pick_sample(s)
            s_next, r, term, trunc, _ = env.step(a)
            done = term or trunc
            buffer.add([s, a, r, s_next, float(term)])
            cum_reward += r

            if buffer.length() >= batch_size:
                states, actions, rewards, n_states, dones = buffer.sample(batch_size)
                q_loss_val, mu_loss_val = optimize(states, actions, rewards, n_states, dones)
                update_target()
                episode_q_loss += q_loss_val
                episode_mu_loss += mu_loss_val
                training_steps += 1

            s = s_next

        # Compute average losses (if at least one training step occurred)
        if training_steps > 0:
            avg_q_loss = episode_q_loss / training_steps
            avg_mu_loss = episode_mu_loss / training_steps
        else:
            avg_q_loss = 0.0
            avg_mu_loss = 0.0

        reward_records.append(cum_reward)
        log_line = f"{ep+1},{cum_reward}\n"
        log_file.write(log_line)
        log_file.flush()
        print(f"Episode {ep+1}: Reward = {cum_reward:.2f}, Avg Q Loss = {avg_q_loss:.6f}, Avg Mu Loss = {avg_mu_loss:.6f}")

    # Save the models after training.
    os.makedirs("saved_models", exist_ok=True)
    torch.save(mu_origin_model.state_dict(), os.path.join("saved_models", "mu_origin_model.pth"))
    torch.save(q_origin_model.state_dict(), os.path.join("saved_models", "q_origin_model.pth"))
    print("Training Done. Models and log saved.")

    log_file.close()

Episode 1: Reward = -355.90, Avg Q Loss = 0.140909, Avg Mu Loss = -0.034331
Episode 2: Reward = -215.27, Avg Q Loss = 0.061065, Avg Mu Loss = 0.210624
Episode 3: Reward = -567.65, Avg Q Loss = 0.190527, Avg Mu Loss = 0.196417
Episode 4: Reward = -403.62, Avg Q Loss = 0.684250, Avg Mu Loss = -0.999314
Episode 5: Reward = -462.16, Avg Q Loss = 0.840658, Avg Mu Loss = -1.286000
Episode 6: Reward = -498.13, Avg Q Loss = 0.707773, Avg Mu Loss = -1.360118
Episode 7: Reward = -530.02, Avg Q Loss = 0.685261, Avg Mu Loss = -1.023902
Episode 8: Reward = -111.63, Avg Q Loss = 0.784151, Avg Mu Loss = -1.081329
Episode 9: Reward = -89.93, Avg Q Loss = 0.915709, Avg Mu Loss = -1.343317
Episode 10: Reward = 204.08, Avg Q Loss = 0.886399, Avg Mu Loss = -1.456991
Episode 11: Reward = -158.10, Avg Q Loss = 0.894964, Avg Mu Loss = -1.730486
Episode 12: Reward = 244.01, Avg Q Loss = 0.916743, Avg Mu Loss = -1.970471
Episode 13: Reward = 276.19, Avg Q Loss = 0.867513, Avg Mu Loss = -2.406944
Episode 14: Re

KeyboardInterrupt: 

In [2]:
!pip install "gymnasium[mujoco]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 18.8 MB/s eta 0:00:00
